# Add metrics to Table

This notebook computes the global image metrics "uniqueness" and "diversity" for each image in a dataset containing images and labels.

![](../images/add-classification-metrics.png)

<!-- Tags: ["Classification", "Metrics", "Diversity", "Uniqueness"] -->

The uniqueness metric quantifies how distinct an image is compared to others within its label group. It is based on the average pairwise distance between the image's embedding and all other embeddings of the same label. Higher uniqueness scores indicate that an image is less similar to others in its group, which can help identify outliers or rare examples within a class.

The diversity metric, on the other hand, ranks images within a label group based on their proximity to the cluster center of that group. Images closer to the cluster center receive lower diversity scores, while those further away are ranked higher. This metric highlights how representative or central an image is to its class, helping to identify examples that are more typical versus those that are more unusual. Together, these metrics provide valuable insights into the structure and variability of the dataset.

## Project setup


## Install dependencies

In [ ]:
%%capture

%pip install 3lc
%pip install git+https://github.com/3lc-ai/3lc-examples.git
%pip install transformers
%pip install pacmap
%pip install joblib

## Imports

In [ ]:
import numpy as np
import tlc

from tlc_tools import add_columns_to_table
from tlc_tools.embeddings import add_embeddings_to_table
from tlc_tools.metrics import diversity, uniqueness

## Load input table

We will reuse the CIFAR-10 table from the notebook [create-table-from-torch.ipynb](../1-create-tables/create-table-from-torch.ipynb)

In [ ]:
table = tlc.Table.from_names("initial", "CIFAR-10-train", "3LC Tutorials - CIFAR-10")

## Add embeddings

In [ ]:
table.map(lambda x: x[0])  # Only return the image

# In order to compute the "uniqueness" and "diversity" metrics, we need embeddings.
# This line adds default embeddings from a Visual Image Transformer model.
added_embeddings_table = add_embeddings_to_table(table)

In [ ]:
reduced_table = tlc.reduce_embeddings(added_embeddings_table, method="pacmap")

In [ ]:
reduced_table.columns

## Compute metrics and add to Table

In [ ]:
label_column_pyarrow = reduced_table.get_column("Label")
label_column = np.array(label_column_pyarrow.to_numpy(zero_copy_only=False).tolist(), dtype=np.int32)

In [ ]:
embeddings_column_pyarrow = reduced_table.get_column("embedding_pacmap")
embeddings_column = np.array(embeddings_column_pyarrow.to_numpy(zero_copy_only=False).tolist(), dtype=np.float32)

In [ ]:
diversity = diversity(label_column, embeddings_column)
uniqueness = uniqueness(label_column, embeddings_column)

In [ ]:
added_columns_table = add_columns_to_table(
    reduced_table,
    columns={
        "image_diversity": diversity,
        "image_uniqueness": uniqueness,
    },
    output_table_name="added-classification-metrics",
)